In [7]:
import os
from pathlib import Path
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv
from datasets import load_dataset, Dataset

import nest_asyncio
nest_asyncio.apply()

In [8]:
dataset_name = load_dataset(os.getenv("HF_HUB_DATASET_NAME"))
assert dataset_name

In [9]:
# select the first 32 examples
sampled_dataset = dataset_name['train'].select(range(2))
len(sampled_dataset)

2

In [10]:
system_message = """
You are given 5 sentences from a story in a shuffled order. 
Each sentence has a serial number assoiciated with it. 
You can find the serial number at the beginning of each sentence, for example: 0. <sentence>.
The serial numbers start from 0 and ends at 4.
Your task is to predict the correct order of sentences which would resemble the original story and then return the serial numbers as a comma separated string.

#IMPORTANT
There are exactly five sentences. 
You should only return the serial numbers as a comma separated string.
"""

client = AsyncOpenAI(
    api_key="token-abc123",
    base_url=os.getenv("VLLM_ENDPOINT_URL")
)

In [22]:
from tqdm.auto import tqdm

formatted_inputs = []
for _, data in tqdm(enumerate(sampled_dataset)):
    ins = ""
    for idx, sent in enumerate(data["shuffled_sentences"]):
        ins += f"{idx}. {sent}\n"
    formatted_inputs.append(ins)
    

0it [00:00, ?it/s]

In [23]:
formatted_inputs[0]

"0. They got themselves and Dan on a diet.\n1. Dan's parents were overweight.\n2. Dan was overweight as well.\n3. His parents understood and decided to make a change.\n4. The doctors told his parents it was unhealthy.\n"

In [24]:
response =  await client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[0]}
    ],
    temperature=0.0,
)


print(response.choices[0].message.content)
print(sampled_dataset[0]["gold_order"])

To predict the correct order of sentences, let's analyze the given sentences:

- Sentence 0 mentions Dan and his family going on a diet, implying that there was a problem that needed to be addressed.
- Sentence 1 and 2 provide the reason for the diet, which is that Dan's parents and Dan himself were overweight.
- Sentence 3 suggests that Dan's parents are taking action to address the issue.
- Sentence 4 provides a reason for the parents' decision to make a change, which is that the doctors told them it was unhealthy.

Based on this analysis, the correct order of sentences would be:

0. They got themselves and Dan on a diet.
1. Dan's parents were overweight.
2. Dan was overweight as well.
3. His parents understood and decided to make a change.
4. The doctors told his parents it was unhealthy.

The serial numbers as a comma-separated string would be: 0,1,2,3,4
[1, 2, 4, 3, 0]


In [25]:
response = await client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[1]}
    ],
    temperature=0.0,
)


print(response.choices[0].message.content)
print(sampled_dataset[1]["gold_order"])

To predict the correct order of sentences, let's analyze the given sentences:

- Sentence 4 mentions that Carrie had just learned how to ride a bike, which suggests that the story is about her learning to ride a bike.
- Sentence 0 mentions Carrie sneaking rides on her sister's bike, which implies that she doesn't have a bike of her own, which is mentioned in sentence 3.
- Sentence 2 mentions Carrie crashing into a wall, which is likely to happen after she has gained some experience and confidence in riding the bike.
- Sentence 1 mentions Carrie getting a deep gash on her leg, which is likely to happen during her early attempts to ride the bike.

Considering these points, the correct order of sentences would be:

4. Carrie had just learned how to ride a bike.
0. Carrie would sneak rides on her sister's bike.
3. She didn't have a bike of her own.
1. The bike frame bent and Carrie got a deep gash on her leg.
2. She got nervous on a hill and crashed into a wall.

The serial numbers as a co

In [27]:
# using instructor
import instructor
from pydantic import BaseModel, Field
from typing import List


class PredictionResult(BaseModel):
    predicted_order: List[int] = Field("A list of integers in the range 0 upto 4, which contains the serial numbers of sentences in the correct order.")


 
iclient = instructor.from_openai(OpenAI(
    api_key="token-abc123",
    base_url=os.getenv("VLLM_ENDPOINT_URL")
))
system_message = """
You are given 5 sentences from a story in a shuffled order. 
Each sentence has a serial number assoiciated with it. 
You can find the serial number at the beginning of each sentence, for example: 0. <sentence>.
The serial numbers start from 0 and ends at 4.
Your task is to predict the correct order of sentences which would resemble the original story and then return the serial numbers as a list of integers, which contain the serial numbers.

#IMPORTANT
There are exactly five sentences. 
"""

In [29]:
response = iclient.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[0]}
    ],
    temperature=0.0,
    response_model=PredictionResult,
)


print(response)
print(sampled_dataset[0]["gold_order"])

predicted_order=[0, 3, 4, 1, 2]
[1, 2, 4, 3, 0]


In [30]:
response = iclient.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[1]}
    ],
    temperature=0.0,
    response_model=PredictionResult,
)


print(response)
print(sampled_dataset[1]["gold_order"])

predicted_order=[4, 0, 1, 2, 3]
[4, 3, 0, 2, 1]


In [33]:
from scipy.stats import kendalltau

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html
kendalltau(x=sampled_dataset[1]["gold_order"], y=response.predicted_order)

SignificanceResult(statistic=np.float64(0.19999999999999998), pvalue=np.float64(0.8166666666666667))

In [34]:
def overlapping_accuracy(gold, predicted):
    assert len(gold) == len(predicted)
    
    overlaps = 0
    for idx in range(len(gold)):
        if gold[idx] == predicted[idx]:
            overlaps += 1
            
    return overlaps / len(gold)


overlapping_accuracy(
    sampled_dataset[1]["gold_order"], response.predicted_order)

0.4